# <center>Computing Euler Angles from a Rotation Matrix</center>

<center>Original paper by: Gregory G. Slabough</center>

## Abstract

This document discusses a simple technique to find all possible Euler angles from a rotation matrix.  Determination of Euler angles is sometimes a necessary step in computer graphics, vision, robotics, and kinematics.  However, the solution may or may not be obvious.

In [9]:
import sys, os
import numpy as np

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=2, width=120)

## Rotation Matrices

We start off with the standard definition of the rotations about the three principle axes.

A rotation of $\psi$ radians about the x-axis is defined as:

$$
R_x(\psi) =
\left[
    \begin{matrix}
    1 & 0 & 0 \\
    0 & cos \, \psi & -sin \, \psi \\
    0 & sin \, \psi & cos \, \psi \\
    \end{matrix}
\right]
$$

Similarly, a rotation of $\theta$ radians about the y-axis is defined as:

$$
R_y(\theta) =
\left[
    \begin{matrix}
    cos \, \theta & 0 & sin \, \theta \\
    0 & 1 & 0 \\
    -sin \, \theta & 0 & cos \, \theta \\
    \end{matrix}
\right]
$$

Finally, a rotation of $\phi$ radians about the z-axis is defined as:

$$
R_y(\phi) =
\left[
    \begin{matrix}
    cos \, \phi & -sin \, \phi & 0 \\
    sin \, \phi & cos \, \phi & 0 \\
    0 & 1 & 0 \\
    \end{matrix}
\right]
$$

The angles $\psi$, $\theta$, and $\phi$ are the Euler angles.  And for some clarification, we associate the common terms Pitch, Yaw, and Roll such that:

$$
\begin{align}
    \psi &= \text{rotation about the x-axis, or Pitch} \\
    \theta &= \text{rotation about the y-axis, or Yaw} \\
    \phi &= \text{rotation about the z-axis, or Roll} \\
\end{align}
$$

## Generalized Rotation Matrices

A general rotation matrix will have the form:

$$
R =
\left[
    \begin{matrix}
    R_{11} & R_{12} & R_{13} \\
    R_{21} & R_{22} & R_{23} \\
    R_{31} & R_{32} & R_{33} \\
    \end{matrix}
\right]
$$

This matrix can be thought of as a sequence of three rotations, one about each principle axis.  Since matrix multiplication does not commute, the order of the axes which one rotates about will affect the result.  For this analysis, we will rotate first about the x-axis, then the y-axis, and finally the z-axis.  Such a sequence of rotations can be represented as the matrix product:

$$
\begin{align}
    R &= R_z(\phi)R_y(\theta)R_x(\psi) \\
    &=
    \left[
        \begin{matrix}
        cos \,\theta \,cos \,\phi  &  sin \,\psi \,sin \,\theta \,cos \,\phi - cos \,\psi \,sin \,\phi  &  cos \,\psi \,sin \,\theta \,cos \,\phi + sin \,\psi \,sin \,\phi \\
        cos \,\theta \,sin \,\phi  &  sin \,\psi \,sin \,\theta \,sin \,\phi + cos \,\psi \,cos \,\phi  &  cos \,\psi \,sin \,\theta \,sin \,\phi - sin \,\psi \,cos \,\phi \\
        -sin \,\theta &  sin \,\psi \,cos \,\theta  &  cos \,\psi \,cos \,\theta \\
        \end{matrix}
    \right] \\
\end{align}
$$

Given a rotation matrix $R$, we can compute the Euler angles $\psi$, $\theta$, and $\phi$ by equating each element in $R$ with the corresponding element in the matrix product $R_z(\phi)R_y(\theta)R_x(\psi)$.  This results in nine equations that can be used to find the Euler angles.

## Finding Two Possible Angles for $\theta$

Starting with $R_{31}$, we find:
$$
R_{31} = -sin \,\theta
$$

This equation can be inverted fo yield:
$$
\theta = -sin^{-1}(R_{31}) \qquad \qquad \qquad (1)
$$

However, one must be careful in interpreting this equation.  Since $sin(\pi - \theta) = sin(\theta)$, there are actually <i>two</i> distinct values (for $R_{31} \ne \pm 1$) of $\theta$ that satisfy Equation 1.  Therefore, both the values:

$$
\begin{align}
    \theta_1 &= -sin^{-1}(R_{31}) \\
    \theta_2 &= \pi - \theta_1 = \pi + sin^{-1}(R_{31})
\end{align}
$$

are valid solutions.  We will handle the special case of $R_{31} = \pm 1$ later.  So using the $R_{31}$ element of the rotation matrix, we are able to determine two possible values for $\theta$.

## Finding the Corresponding Angles of $\psi$

To find the values for $\psi$, we observe that:
$$
    {R_{32} \over R_{33}} = tan(\psi)
$$

We use this equation to solve for $\psi$ as:
$$
    \psi = atan2(R_{32}, R_{33}) \qquad \qquad \qquad (2)
$$

where $atan2(y, x)$ is the arc tangent of the two variables $x$ and $y$.  It is similar to calculating the arc tangent of $y/x$, except that the signs of both arguments are used to determine the quadrant of the result, which lies in the range $[-\pi, \pi]$.  The function atan2 is available in many programming languages.

One must be careful in interpreting Equation 2.  If $cos(\theta) > 0$, then $\psi = atan2(R_{32}, R_{33})$.  However, when $cos(\theta) < 0$, then $\psi = atan2(-R_{32}, -R_{33})$.  A simple way to handle this is to use the equation:
$$
    \psi = atan2 \left({R_{32} \over cos \theta}, {R_{33} \over cos \theta} \right) \qquad \qquad \qquad (3)
$$

Equation 3 is valid for all cases except when $cos(\theta) = 0$.  We will deal with this special case later.  For each value of $\theta$, we compute a corresponding value of $\psi$ using Equation 3, yielding:
$$
\begin{align}
    \psi_1 &= atan2 \left({R_{32} \over cos \theta_1}, {R_{33} \over cos \theta_1} \right) \qquad \qquad \qquad (4) \\
    \psi_2 &= atan2 \left({R_{32} \over cos \theta_2}, {R_{33} \over cos \theta_2} \right) \qquad \qquad \qquad (5)
\end{align}
$$

## Finding the Corresponding Angles of $\phi$

A similar analysis holds for finding $\phi$.  We observe that:
$$
{R_{22} \over R_{11}} = tan \phi
$$

We solve for $\phi$ using the equation:
$$
\phi = atan2 \left({R_{21} \over cos \phi}, {R_{11} \over cos \phi} \right) \qquad \qquad \qquad (6)
$$

Again, this equation is valid for all cases <i>except</i> when $cos \phi = 0$.  And again, we will deal with this special case later.  For each value of $\theta$, we compute a corresponding value of $\phi$ using Equation 6 as:
$$
\begin{align}
    \phi_1 &= atan2 \left({R_{21} \over cos \theta_1}, {R_{11} \over cos \theta_1} \right) \qquad \qquad \qquad (7) \\
    \phi_2 &= atan2 \left({R_{21} \over cos \theta_2}, {R_{11} \over cos \theta_2} \right) \qquad \qquad \qquad (8)
\end{align}
$$

## Two Solutions if $cos \theta \ne 0$

For the case of $cos \theta \ne 0$, we now have two triplets of Euler angles that reproduce the rotation matrix, namely:

$$
(\psi_1, \theta_1, \phi_1) \\
(\psi_2, \theta_2, \phi_2) \\
$$

<i>Both</i> of these solutions will be valid.

## What if $cos \theta = 0$?

This technique described above does not work if the $R_{31}$ element of the rotation matrix is 1 or -1, which corresponds to $\theta = -\pi/2$ or $\theta = \pi/2$, respectively, and to $cos \theta = 0$.  When we try to solve for the possible values of $\psi$ and $\phi$ using the above tecnique, problems will occur, since the elements $R_{11}$, $R_{21}$, $R_{32}$, and $R_{33}$ will all be zero, and therefore Equations 3 and 6 will become:
$$
\begin{align}
    \psi &= atan2 \left( {0 \over 0}, {0 \over 0} \right) \\
    \phi &= atan2 \left( {0 \over 0}, {0 \over 0} \right)
\end{align}
$$

In this case $R_{11}$, $R_{21}$, $R_{32}$, and $R_{33}$ do not constrain the values of $\psi$ and $\phi$.  Therefore, we must use different elements of the rotation matrix to compute the values of $\psi$ and $\phi$.

### <center>Case: $\theta = \pi / 2$</center>

Consider the case when $\theta = \pi / 2$.  Then:

$$
\begin{align}
    R_{12} &= sin \,\psi \, cos \,\phi - cos \,\psi \,sin \,\phi = sin(\psi - \phi) \\
    R_{13} &= cos \,\psi \, cos \,\phi + sin \,\psi \,sin \,\phi = cos(\psi - \phi) \\
    R_{22} &= sin \,\psi \, sin \,\phi + cos \,\psi \,cos \,\phi = cos(\psi - \phi) = R_{13} \\
    R_{23} &= cos \,\psi \, sin \,\phi - sin \,\psi \,cos \,\phi = -sin(\psi - \phi) = -R_{12} \\
\end{align}
$$

Any $\psi$ and $\phi$ that satisfy these equations will be a valid solution.  Using the equations for $R_{12}$ and $R_{13}$, we find that:

$$
\begin{align}
    (\psi - \phi) &= atan2(R_{12}, R_{13}) \\
    \psi &= \phi + atan2(R_{12}, R_{13}) \\
\end{align}
$$

### <center>Case: $\theta = -\pi / 2$</center>

Not surprisingly, a similar result holds for the case when $\theta = -\pi / 2$, for which:

$$
\begin{align}
    R_{12} &= -sin \,\psi \, cos \,\phi - cos \,\psi \,sin \,\phi = -sin(\psi - \phi) \\
    R_{13} &= -cos \,\psi \, cos \,\phi + sin \,\psi \,sin \,\phi = -cos(\psi - \phi) \\
    R_{22} &= -sin \,\psi \, sin \,\phi + cos \,\psi \,cos \,\phi = cos(\psi - \phi) = -R_{13} \\
    R_{23} &= -cos \,\psi \, sin \,\phi - sin \,\psi \,cos \,\phi = -sin(\psi - \phi) = R_{12} \\
\end{align}
$$

Again, using the equations for $R_{12}$ and $R_{13}$, we find that:

$$
\begin{align}
    (\psi + \phi) &= atan2(-R_{12}, -R_{13}) \\
    \psi &= -\phi + atan2(-R_{12}, -R_{13}) \\
\end{align}
$$

### <center>In Either Case:</center>

In both the $\theta = \pi / 2$ and $\theta = -\pi / 2$ cases, we have found that $\psi$ and $\phi$ are linked.  This phenomenon is called <i>Gimbal Lock</i>.  Although in this case, there are an infinite number of solutions to the problem, in practice, one is often interested in finding one solution.  For this task, it is convenient to set $\phi = 0$ and compute $\psi$ as described above.

## A Pseudo-code Example

We now summarize the method by providing a Python language implementation.  Python syntax is very readable, and is appropriate for many if not most pseudo-code descriptions.

In [24]:
def euler_angles_from_rotation_mx(rot):
    '''
        rot = 3x3 rotation matrix
              index values start at 0, not 1
    '''
    theta_1 = theta_2 = None
    psi_1 = psi_2 = None
    phi_1 = phi_2 = None

    if rot[2, 0] in (1.0, -1.0):
        phi_1 = 0.0  # can be anything

        if rot[2, 0] == -1.0:
            theta_1 = np.pi / 2.0
            psi_1 = phi_1 + np.arctan2(rot[0, 1], rot[0, 2])
        else:
            theta_1 = -np.pi / 2.0
            psi_1 = -phi_1 + np.arctan2(-rot[0, 1], -rot[0, 2])
    else:
        theta_1 = -np.arcsin(rot[2, 0])
        theta_2 = np.pi - theta_1

        psi_1 = np.arctan2(rot[2, 1] / np.cos(theta_1),
                           rot[2, 2] / np.cos(theta_1))
        psi_2 = np.arctan2(rot[2, 1] / np.cos(theta_2),
                           rot[2, 2] / np.cos(theta_2))

        phi_1 = np.arctan2(rot[1, 0] / np.cos(theta_1),
                           rot[0, 0] / np.cos(theta_1))
        phi_2 = np.arctan2(rot[1, 0] / np.cos(theta_2),
                           rot[0, 0] / np.cos(theta_2))

    return [psi_1, theta_1, phi_1], [psi_2, theta_2, phi_2]



## A Computational Example Using Real Data

Suppose we are asked to find the Euler angles that produce the matrix:

$$
R =
\left[
    \begin{matrix}
    .5      &  -.1464  &  .8536 \\
    .5      &  .8536   &  -.1464 \\
    -.7071  &  .5      &  .5 \\
    \end{matrix}
\right]
$$

First, we find the possible values for $\theta$ to be:

$$
\begin{align}
    \theta_1 = -sin(-.7071) = {\pi \over 4} \\
    \theta_2 = \pi - \theta_1 = {3\pi \over 4} \\
\end{align}
$$

Then, we find the corresponding values of $\psi$ to be:

$$
\begin{align}
    \psi_1 &= atan2 \left( {.5 \over cos(\pi / 4)}, {.5 \over cos(\pi / 4)} \right) = {\pi \over 4} \\
    \psi_2 &= atan2 \left( {.5 \over cos(3 \pi / 4)}, {.5 \over cos(3 \pi / 4)} \right) = -{3 \pi \over 4}
\end{align}
$$

And we find $\phi$ to be:

$$
\begin{align}
    \phi_1 &= atan2 \left( {.5 \over cos(\pi / 4)}, {.5 \over cos(\pi / 4)} \right) = {\pi \over 4} \\
    \phi_2 &= atan2 \left( {.5 \over cos(3 \pi / 4)}, {.5 \over cos(3 \pi / 4)} \right) = -{3 \pi \over 4}
\end{align}
$$

Therefore, the solutions are:

$$
\begin{align}
    &\left({\pi \over 4}, {\pi \over 4}, {\pi \over 4} \right) \\
    &\left(-{3 \pi \over 4}, {3 \pi \over 4}, -{3 \pi \over 4} \right)
\end{align}
$$

Let's see if this actually pans out in our code example:

In [25]:
R = np.array([[0.5,    -0.1464, 0.8536],
              [0.5,     0.8536, -0.1464],
              [-0.7071, 0.5,    0.5]])


print 'pi/4.0: ', np.pi / 4.0
print '3*pi/4.0: ', 3.0 * np.pi / 4.0

print
print 'our angles (psi, theta, phi):'
pp.pprint(euler_angles_from_rotation_mx(R))


pi/4.0:  0.785398163397
3*pi/4.0:  2.35619449019

our angles (psi, theta, phi):
( [0.78539816339744828, 0.78538857339744761, 0.78539816339744828],
  [-2.3561944901923448, 2.3562040801923456, -2.3561944901923448])


Yes, it does appear that our code example produces the results that we expected.
Our function returns two vectors of the form $(\psi, \theta, \phi)$ which are equivalent to the more modern terms $(Pitch, Yaw, Roll)$.

## More Than One Solution??

It is interesting to note that there is <i>always</i> more than one sequence of rotations about the three principle axes that results in the same orientation of an object.  As we have shown, in the non-degenerate case of $cos \theta \ne 0$, there are two solutions.  For the degenerate case of $cos \theta = 0$, an infinite number of solutions exist.

As an example, consider a book laying on a table face up in front of you.  Define the x-axis as to the right, the y-axis as away from you, and the z-axis up.  A rotation of $\pi$ radians about the y-axis will turn the book so that the back cover is now facing up.  Another way to achieve the same orientation would be to rotate the book $\pi$ radians about the x-axis, and then $\pi$ radians about the z-axis.  Thus, there is more than one way to achieve a desired rotation.

## Notes

In [26]:
R = np.array([[0.707107,    0.0, -0.707107],
              [-0.0,     1.0, 0.0],
              [0.707107, 0.0,    0.707107]])


print 'pi/4.0: ', np.pi / 4.0
print '3*pi/4.0: ', 3.0 * np.pi / 4.0

print
print 'our angles (psi, theta, phi):'
pp.pprint(euler_angles_from_rotation_mx(R))


pi/4.0:  0.785398163397
3*pi/4.0:  2.35619449019

our angles (psi, theta, phi):
([0.0, -0.78539847284644837, -0.0], [-3.1415926535897931, 3.9269911264362416, 3.1415926535897931])
